In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings

In [26]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [27]:
# Importing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [28]:
%run features_testing.ipynb

In [29]:
# Split the data into explanatory and target variables
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
X_test = test.values

In [30]:
#from sklearn.preprocessing import PowerTransformer
#
#pt = PowerTransformer(method='yeo-johnson')
#X_pt = pt.fit_transform(X)
#X_test_pt = pt.transform(X_test)

In [31]:
# Standardization
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
X_test_std = sc.transform(X_test)

In [32]:
# Split the original data into the training data and the validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_std, y, test_size=0.3, stratify=y, random_state=0)

In [33]:
lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "learning_rate": 0.03,

    "num_leaves": 16,          
    "max_depth": 4,              
    "min_child_samples": 40,
    "min_child_weight": 1e-3,

    "subsample": 0.8,
    "subsample_freq": 1,
    "colsample_bytree": 0.7,

    "reg_alpha": 0.4,    
    "reg_lambda": 0.6,

    "n_estimators": 2000,
    "early_stopping_rounds": 200,
    "random_state": 42,
    "verbose": -1
}

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",

        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", -1, 12),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),

        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),

        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),

        "n_estimators": trial.suggest_int("n_estimators", 200, 3000),
        "verbose": -1,

        "device": "gpu",           
        "gpu_platform_id": 0,      
        "gpu_device_id": 0,        
    }

    n_splits = 10
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_std, y)):
        X_train_fold = X_std[train_idx]
        y_train_fold = y[train_idx]
        X_val_fold = X_std[val_idx]
        y_val_fold = y[val_idx]
        
        model = lgb.LGBMClassifier(**params)
        
        model.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=0)
            ]
        )
        
        preds = model.predict_proba(X_val_fold)[:, 1]
        fold_auc = roc_auc_score(y_val_fold, preds)
        auc_scores.append(fold_auc)
        
        print(f"Fold {fold + 1} AUC: {fold_auc:.6f}")
    
    # why return mean ???
    # Return mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    print(f"Mean AUC: {mean_auc:.6f} (+/- {std_auc:.6f})\n")
    
    return mean_auc


# Create and run the study
study = optuna.create_study(
    direction="maximize",
    study_name="lgbm_optimization_kfold"
)

study.optimize(objective, n_trials=50)

print("\nBest Mean AUC:", study.best_value)
print("Best hyperparameters:\n", study.best_params)

[I 2025-11-25 23:05:38,952] A new study created in memory with name: lgbm_optimization_kfold
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[564]	valid_0's auc: 0.73906


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 1 AUC: 0.739060
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[649]	valid_0's auc: 0.747192


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

Fold 5 AUC: 0.758083
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1189]	valid_0's auc: 0.751774


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 6 AUC: 0.751774
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1690]	valid_0's auc: 0.743195


/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 7 AUC: 0.743195
Training until validation scores don't improve for 100 rounds


In [ ]:
# lightGBM cause seem like this is the best for the old model
import lightgbm as lgb
# optuna para 
hyperpara = {'num_leaves': 57, 'max_depth': 4, 'learning_rate': 0.03277842323048595, 'feature_fraction': 0.9569717768433371, 'bagging_fraction': 0.8506767379185446, 'bagging_freq': 1, 'min_child_samples': 48, 'lambda_l1': 0.17636584028657937, 'lambda_l2': 0.025244809309038312, 'n_estimators': 2486}

lgbm = lgb.LGBMClassifier(**hyperpara)
lgbm.fit(X_train, y_train, eval_set= [(X_valid, y_valid)])

lgbm_train_pred = lgbm.predict_proba(X_train)[:, 1]
lgbm_valid_pred = lgbm.predict_proba(X_valid)[:, 1]

print(f"Train Score: {roc_auc_score(y_train, lgbm_train_pred)}")
print(f"Valid Score: {roc_auc_score(y_valid, lgbm_valid_pred)}")

/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Train Score: 0.8609030994382404
Valid Score: 0.7439516935057933


### under this is the output part


In [ ]:
# Make predictions for the test data
# Change model name if needed
pred = lgbm.predict_proba(X_test_std)[:, 1]

/home/tuanhung/Desktop/Dev/competitionAi/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# Put the prediction into the format of submission
sample_sub['TARGET'] = pred
sample_sub

,SK_ID_CURR,TARGET
0,171202,0.018494
1,171203,0.087814
2,171204,0.214746
3,171205,0.074627
4,171206,0.150592
...,...,...
61495,232697,0.215641
61496,232698,0.040867
61497,232699,0.032599
61498,232700,0.081005


In [ ]:
# Create the "output" directory if it doesn't exist
output_dir = Path.cwd() / "output"
os.makedirs(output_dir, exist_ok=True)

# Specify the new output file path
output_file = output_dir / "submission.csv"

# Save the CSV file to the "output" directory
sample_sub.to_csv(output_file, index=False)